# Getting started with DeepMatcher

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/sidharthms/deepmatcher/blob/master/examples/getting_started.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly perform entity matching using deep neural networks. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train our neural network using supervised learning. At the end of this tutorial, you will have a trained neural network as output which you can easily apply to unlabeled tuple pairs to make predictions.

As an overview, here are the 4 steps to use `deepmatcher` which we will go through in this tutorial:

<ol start="0">
  <li>Setup</li>
  <li>Process labeled data</li>
  <li>Define neural network model</li>
  <li>Train model</li>
  <li>Apply model to new data</li>
</ol>

Let's begin!

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [1]:
try:
    import deepmatcher
except:
    !pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.1-cp36-cp36m-linux_x86_64.whl
    !pip install -q --process-dependency-links git+https://github.com/sidharthms/deepmatcher

Now let's import `deepmatcher` which will do all the heavy lifting to build and train neural network models for entity matching. 

In [2]:
import deepmatcher as dm

We recommend having a GPU available for the training in Step 4. In case a GPU is not available, we will use all available CPU cores. You can run the following command to determine if a GPU is available and will be used for training:

In [3]:
import torch
torch.cuda.is_available()

True

### Download sample data for entity matching

Now let's get some sample data to play with in this tutorial. We will need three sets of labeled data and one set of unlabeled data:

1. **Training Data:** This is used for training our neural network model.
2. **Validation Data:** This is used for determining the configuration (i.e., hyperparameters) of our model in such a way that the model does not overfit to the training set.
3. **Test Data:** This is used to estimate the performance of our trained model on unlabeled data.
4. **Unlabeled Data:** The trained model is applied on this data to obtain predictions, which can then be used for downstream tasks in practical application scenarios.

We download these four data sets to the `sample_data` directory:

In [4]:
!mkdir -p sample_data/itunes-amazon
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/itunes-amazon/train.csv
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/itunes-amazon/validation.csv
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/itunes-amazon/test.csv
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/itunes-amazon/unlabeled.csv

To get an idea of how our data looks like, let's take a peek at the training dataset:

In [5]:
import pandas as pd
pd.read_csv('sample_data/itunes-amazon/train.csv').head()

,id,left_id,right_id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,5073,34877,0,Baby When the Light ( David Guetta & Fred Rist...,David Guetta,Pop Life ( Extended Version ) [ Bonus Version ],"Dance , Music , Rock , Pop , House , Electroni...",$ 1.29,‰ ãÑ 2007 Gum Records,6:17,18-Sep-07,Revolver ( Madonna Vs. David Guetta Feat . Lil...,David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,3:18,"August 21 , 2009"
1,287,2834,53255,1,Outversion,Mark Ronson,Version,"Pop , Music , R&B / Soul,Soul,Dance,Rock,Jazz,...",$ 0.99,2007 Mark Ronson under exclusive license to SO...,1:50,10-Jul-07,Outversion,Mark Ronson,Version [ Explicit ],Pop,$ 0.99,( c ) 2011 J'adore Records,1:50,"July 10 , 2007"
2,534,6866,13587,0,Peer Pressure ( feat . Traci Nelson ),Snoop Dogg,Doggumentary,"Hip-Hop/Rap , Music , Rock , Gangsta Rap , Wes...",$ 1.29,"‰ ãÑ 2011 Capitol Records , LLC . All rights r...",4:07,29-Mar-11,Boom ( ( Feat . T-Pain ) [ Edited ] ),Snoop Dogg,Doggumentary [ Edited ],"Rap & Hip-Hop , West Coast",$ 1.29,"( C ) 2011 Capitol Records , LLC",3:50,"March 29 , 2011"
3,181,1358,42278,1,Stars Come Out ( Tim Mason Remix ),Zedd,Stars Come Out ( Remixes ) - EP,"Dance , Music , Electronic , House",$ 1.29,2012 Dim Mak Inc.,5:49,20-May-14,Stars Come Out ( Dillon Francis Remix ),Zedd,Stars Come Out [ Dillon Francis Remix ],Dance & Electronic,$ 1.29,2012 Dim Mak Inc.,4:08,"May 20 , 2014"
4,485,5695,37659,0,Jump ( feat . Nelly Furtado ),Flo Rida,R.O.O.T.S. ( Deluxe Version ),"Hip-Hop/Rap , Music",$ 1.29,‰ ãÑ 2009 Atlantic Recording Corporation for t...,3:28,30-Mar-09,"Yayo [ Feat . Brisco , Billy Blue , Ball Greez...",Flo Rida,R.O.O.T.S. ( Route Of Overcoming The Struggle ...,Rap & Hip-Hop,$ 1.29,"( C ) 2012 Motown Records , a Division of UMG ...",7:53,"March 30 , 2009"


## Step 1. Process labeled data

Before we can use our data for training, `deepmatcher` needs to first load and process it in order to prepare it for neural network training. Currently `deepmatcher` only supports processing CSV files. Each CSV file is assumed to have the following kinds of columns:

* **"Left" attributes (required):** Our goal is to match tuple pairs. "Left" attributes are columns that correspond to the "left" tuple or the first tuple in the tuple pair. These column names are expected to be prefixed with "left_" by default.
* **"Right" attributes (required):** "Right" attributes are columns that correspond to the "right" tuple or the second tuple in the tuple pair. These column names are expected to be prefixed with "right_" by default.
* **Label column (required for train, validation, test):** Column containing the labels (match or non-match) for each tuple pair. Expected to be named "label" by default
* **ID column (required):** Column containing a unique ID for each tuple pair. This is for evaluation convenience.  Expected to be named "id" by default.

More details on what data processing involves and ways to customize it are described in **[this notebook](https://nbviewer.jupyter.org/github/sidharthms/deepmatcher/blob/master/examples/data_processing.ipynb)**. 

### Processing labeled data
In order to process our train, validation and test CSV files we call `dm.data.process` in the following code snippet which will load and process the CSV files and return three processed `MatchingDataset` objects respectively. These dataset objects will later be used for training and evaluation. The basic parameters to `dm.data.process` are as follows:

* **path (required): ** The path where all data is stored. This includes train, validation and test. `deepmatcher` may create new files in this directory to store information about these data sets. This allows subsequent `dm.data.process` calls to be much faster.
* **train (required): ** File name of training data in `path` directory.
* **validation (required): ** File name of validation data in `path` directory.
* **test (optional): ** File name of test data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV files that you may want to ignore for the purposes of training. These should be included here. 

Note that the train, validation and test CSVs must all share the same schema, i.e., they should have the same columns. Processing data involves several steps and can take several minutes to complete, especially if this is the first time you are running the `deepmatcher` package.

NOTE: If you are running this in Colab, you may get a message saying 'Memory usage is close to the limit.' You can safely ignore it for now. We are working on reducing the memory footprint.

In [6]:
train, validation, test = dm.data.process(
    path='sample_data/itunes-amazon',
    train='train.csv',
    validation='validation.csv',
    test='test.csv',
    ignore_columns=('left_id', 'right_id'))

INFO:deepmatcher.data.field:Downloading vectors from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh


Load time: 1.1070073630000365
downloading from Google Drive; may take a few minutes


/usr/local/lib/python3.6/dist-packages/deepmatcher/data/field.py:62: ResourceWarning: unclosed <ssl.SSLSocket fd=64, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.2', 44956), raddr=('74.125.197.138', 443)>
  download_from_url(url, self.destination)
/usr/local/lib/python3.6/dist-packages/deepmatcher/data/field.py:62: ResourceWarning: unclosed <ssl.SSLSocket fd=65, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.2', 46304), raddr=('74.125.197.132', 443)>
  download_from_url(url, self.destination)
INFO:deepmatcher.data.field:Extracting vectors into /content/.vector_cache


Vocab time: 117.61075533899998
Metadata time: 0.7012989679999464
Cache time: 0.7771342040000491


#### Peeking at processed data
Let's take a look at how the processed data looks like. To do this, we get the raw `pandas` table corresponding to the processed training dataset object. 

In [7]:
train_table = train.get_raw_table()
train_table.head()

,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,baby when the light ( david guetta &amp; fred ...,david guetta,pop life ( extended version ) &#91; bonus vers...,"dance , music , rock , pop , house , electroni...",$ 1.29,‰ ãñ 2007 gum records,6 : 17,18-sep-07,revolver ( madonna vs . david guetta feat . li...,david guetta,one love ( deluxe version ),dance &amp; electronic,$ 1.29,( c ) 2014 swedish house mafia holdings ltd ( ...,3 : 18,"august 21 , 2009"
1,287,1,outversion,mark ronson,version,"pop , music , r &amp; b / soul , soul , dance ...",$ 0.99,2007 mark ronson under exclusive license to so...,1 : 50,10-jul-07,outversion,mark ronson,version &#91; explicit &#93;,pop,$ 0.99,( c ) 2011 j &apos;adore records,1 : 50,"july 10 , 2007"
2,534,0,peer pressure ( feat . traci nelson ),snoop dogg,doggumentary,"hip-hop / rap , music , rock , gangsta rap , w...",$ 1.29,"‰ ãñ 2011 capitol records , llc . all rights r...",4 : 07,29-mar-11,boom ( ( feat . t-pain ) &#91; edited &#93; ),snoop dogg,doggumentary &#91; edited &#93;,"rap &amp; hip-hop , west coast",$ 1.29,"( c ) 2011 capitol records , llc",3 : 50,"march 29 , 2011"
3,181,1,stars come out ( tim mason remix ),zedd,stars come out ( remixes ) - ep,"dance , music , electronic , house",$ 1.29,2012 dim mak inc .,5 : 49,20-may-14,stars come out ( dillon francis remix ),zedd,stars come out &#91; dillon francis remix &#93;,dance &amp; electronic,$ 1.29,2012 dim mak inc .,4 : 08,"may 20 , 2014"
4,485,0,jump ( feat . nelly furtado ),flo rida,r.o.o.t.s . ( deluxe version ),"hip-hop / rap , music",$ 1.29,‰ ãñ 2009 atlantic recording corporation for t...,3 : 28,30-mar-09,"yayo &#91; feat . brisco , billy blue , ball g...",flo rida,r.o.o.t.s . ( route of overcoming the struggle...,rap &amp; hip-hop,$ 1.29,"( c ) 2012 motown records , a division of umg ...",7 : 53,"march 30 , 2009"


The processed attribute values have been tokenized and lowercased so they may not look exactly the same as the input training data. These modifications help the neural network generalize better, i.e., perform better on data not trained on. 

## Step 2. Define neural network model

In this step you tell `deepmatcher` what kind of neural network you would like to use for entity matching. The easiest way to do this is to use one of the several kinds of neural network models that comes built-in with `deepmatcher`. To use a built-in network, construct a `dm.MatchingModel` as follows:

`model = dm.MatchingModel(attr_summarizer='<TYPE>')`

where `<TYPE>` is one of `sif`, `rnn`, `attention` or `hybrid`. If you are not familiar with what these mean, we strongly recommend taking a look at either **[slides from our talk on deepmatcher](http://bit.do/deepmatcher-talk)** for a high level overview, or **[our paper](http://pages.cs.wisc.edu/~anhai/papers1/deepmatcher-sigmod18.pdf)** for a more detailed explanation. Here we give briefly describe the intuition behind these four model types:
* **sif:** This model considers the **words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **rnn:** This model considers the **sequences of words** present in each attribute value pair to determine a match or non-match.
* **attention:** This model considers the **alignment of words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **hybrid:** This model considers the **alignment of sequences of words** present in each attribute value pair to determine a match or non-match.

`deepmatcher` is highly customizable and allows you to tune almost every aspect of the neural network model for your application scenario. **[This tutorial](https://nbviewer.jupyter.org/github/sidharthms/deepmatcher/blob/master/examples/matching_models.ipynb)** discusses the structure of `MatchingModel`s and how they can be customized.

For this tutorial, let's create a `hybrid` model for entity matching:

In [8]:
model = dm.MatchingModel(attr_summarizer='hybrid')

## Step 3. Train model

Next, we train the defined neural network model using the processed training and validation data. To do so, we call the `run_train` method which takes the following basic parameters:

* **train:** The processed training dataset object (of type `MatchingDataset`).
* **validation:** The processed validation dataset object (of type `MatchingDataset`).
* **epochs:** Number of times to go over the entire `train` data for training the model.
* **batch_size:** Number of labeled examples (tuple pairs) to use for each training step. This value may be increased if you have a lot of training data and would like to speed up training. The optimal value is dataset dependent.
* **best_save_path:** Path to save the best model.
* **pos_neg_ratio**: The ratio of the weight of positive examples (matches) to weight of negative examples (non-matches). This value should be increased if you have fewer matches than non-matches in your data. The optimal value is dataset dependent.

Many other aspects of the training algorithm can be customized. For details on this, please refer the API documentation for **[run_train]()**

In [9]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1 :


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 1 || Run Time:   14.4 | Load Time:    2.6 || F1:  46.84 | Prec:  45.68 | Rec:  48.05 || Ex/s:  19.09

===>  EVAL Epoch 1 :


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    1.7 | Load Time:    0.8 || F1:  72.73 | Prec:  64.52 | Rec:  83.33 || Ex/s:  43.03

* Best F1: 72.72727272727272
Saving best model...
===>  TRAIN Epoch 2 :


...

Finished Epoch 14 || Run Time:    1.7 | Load Time:    0.8 || F1:  89.80 | Prec:  88.00 | Rec:  91.67 || Ex/s:  42.68

===>  TRAIN Epoch 15 :


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:   13.8 | Load Time:    2.6 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  19.70

===>  EVAL Epoch 15 :


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 15 || Run Time:    1.7 | Load Time:    0.8 || F1:  89.80 | Prec:  88.00 | Rec:  91.67 || Ex/s:  43.21

Loading best model...


## Step 4. Apply model to new data

### Evaluating on test data
Now that we have a trained model for entity matching, we can now evaluate its accuracy on test data, to estimate the performance of the model on unlabeled data.

In [10]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 7 :
Finished Epoch 7 || Run Time:    1.0 | Load Time:    0.8 || F1:  84.21 | Prec:  92.31 | Rec:  77.42 || Ex/s:  60.42



84.21052631578948

### Evaluating on unlabeled data

We finally apply the trained model to unlabeled data to get predictions. To do this, we need to first process the unlabeled data.

#### Processing unlabeled data

To process unlabeled data, we use `dm.data.process_unlabeled`, as shown in the code snippet below. The basic parameters for this call are as follows:

* **path (required): ** The full path to the unlabeled data file (not just the directory).
* **trained_model (required): ** The trained model. The model is aware of the configuration of the training data on which it was trained, and so `deepmatcher` reuses the same configuration for the unlabeled data.
* **ignore_columns (optional): ** Any columns in the unlabeled CSV file that you may want to ignore for the purposes of training. If not specified, the columns that were ignored while processing the training set will also be ignored while processing the unlabeled data.

Note that the unlabeled CSV file must have the same schema as the train, validation and test CSVs.

In [11]:
unlabeled = dm.data.process_unlabeled(
    path='sample_data/itunes-amazon/unlabeled.csv',
    trained_model=model,
    ignore_columns=('left_id', 'right_id'))

Load time: 0.43851208899991434
Vocab update time: 0.02722389599989583


#### Obtaining predictions

Next, we call the `run_prediction` method which takes a processed data set object and returns a `pandas` dataframe containing tuple pair IDs (`id` column) and the corresponding match score predictions (`match_score` column). `match_scores` are in [0, 1] and a score above 0.5 indicates a match prediction.

In [12]:
predictions = model.run_prediction(unlabeled)
predictions.head()

===>  PREDICT Epoch 7 :


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 7 || Run Time:    2.5 | Load Time:    1.5 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



,match_score
id,
28998,0.120477
29266,0.059813
29265,0.059813
29262,0.059813
29261,0.059813


You may optionally set the `output_attributes` parameter to also include all attributes present in the original input table. As mentioned earlier, the processed attribute values will likely look a bit different from the attribute values in the input CSV files due to modifications such as tokenization and lowercasing.

In [13]:
predictions = model.run_prediction(unlabeled, output_attributes=True)
predictions.head()

===>  PREDICT Epoch 7 :


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 7 || Run Time:    2.4 | Load Time:    1.4 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



,match_score,left_id,right_id,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
id,,,,,,,,,,,,,,,,,,,
28998,0.120477,2298,1235,Because of Your Love,Kenny Chesney,Greatest Hits,"Country , Music , Contemporary Country , Urban...",$ 0.99,2000 BMG Entertainment,4:16,26-Sep-00,Run Thru Your Mind,Kenny Rogers,49 All Time Greatest Hits,"Country , Traditional Country , Cowboy",$ 0.89,( c ) 2008 Goldenlane Records,2:57,"August 1 , 2008"
29266,0.059813,2482,1238,Because of Your Love,Kenny Chesney,Greatest Hits,"Country , Music , Contemporary Country , Urban...",$ 0.99,2000 BMG Entertainment,4:16,26-Sep-00,Hurry Up Love,Kenny Rogers,49 All Time Greatest Hits,"Country , Traditional Country , Cowboy",$ 0.89,( c ) 2008 Goldenlane Records,2:29,"August 1 , 2008"
29265,0.059813,2477,1238,Because of Your Love,Kenny Chesney,Greatest Hits,"Country , Music , Contemporary Country , Urban...",$ 0.99,2000 BMG Entertainment,4:16,26-Sep-00,Hurry Up Love,Kenny Rogers,49 All Time Greatest Hits,"Country , Traditional Country , Cowboy",$ 0.89,( c ) 2008 Goldenlane Records,2:29,"August 1 , 2008"
29262,0.059813,2370,1238,Because of Your Love,Kenny Chesney,Greatest Hits,"Country , Music , Contemporary Country , Urban...",$ 0.99,2000 BMG Entertainment,4:16,26-Sep-00,Hurry Up Love,Kenny Rogers,49 All Time Greatest Hits,"Country , Traditional Country , Cowboy",$ 0.89,( c ) 2008 Goldenlane Records,2:29,"August 1 , 2008"
29261,0.059813,2369,1238,Because of Your Love,Kenny Chesney,Greatest Hits,"Country , Music , Contemporary Country , Urban...",$ 0.99,2000 BMG Entertainment,4:16,26-Sep-00,Hurry Up Love,Kenny Rogers,49 All Time Greatest Hits,"Country , Traditional Country , Cowboy",$ 0.89,( c ) 2008 Goldenlane Records,2:29,"August 1 , 2008"


You can then save these predictions to CSV and use them for downstream tasks.

In [14]:
predictions.to_csv('sample_data/itunes-amazon/unlabeled_predictions.csv')

#### Getting predictions on labeled data

You can also get predictions for labeled data such as validation data. To do so, you can simply call the `run_prediction` method passing the validation data as argument.

In [15]:
valid_predictions = model.run_prediction(validation, output_attributes=True)
valid_predictions.head()

===>  PREDICT Epoch 7 :
Finished Epoch 7 || Run Time:    1.0 | Load Time:    0.8 || F1:  91.67 | Prec:  91.67 | Rec:  91.67 || Ex/s:  58.60



,match_score,left_id,right_id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
id,,,,,,,,,,,,,,,,,,,,
180,0.275759,1355,6230,0,Stars Come Out ( Tim Mason Remix ),Zedd,Stars Come Out ( Remixes ) - EP,"Dance , Music , Electronic , House",$ 1.29,2012 Dim Mak Inc.,5:49,20-May-14,Beautiful Now ( Marshmello Remix ) [ feat . Jo...,Zedd,Beautiful Now ( Remixes ),Dance & Electronic,$ 1.29,( C ) 2015 Interscope Records,3:16,"July 10 , 2015"
240,0.174738,2022,28261,0,If I Am a Stranger ( Live In Amsterdam ),Ryan Adams,Live After Deaf ( Live ) [ Complete Collection ],"Singer/Songwriter , Music , Rock",$ 0.99,2012 PAXAM RECORDS,5:10,2-Jul-12,The Rescue Blues ( Live in London 1 ),Ryan Adams,Live After Deaf ( Collection ),"Folk , Rock",$ 0.99,2012 PAXAM RECORDS,3:25,"July 2 , 2012"
295,0.062724,2959,7497,0,Thinking About You ( feat . Ayah Marar ) [ Mic...,Calvin Harris,Thinking About You ( feat . Ayah Marar ) [ Rem...,"Dance , Music",$ 1.29,2013 Sony Music Entertainment UK Limited,5:42,2-Aug-13,Blame ( Burns Remix ),Calvin Harris feat . John Newman,Blame ( Remixes ),Dance & Electronic,$ 1.29,2013 1i Music,4:17,"November 10 , 2014"
231,0.109148,2020,28191,0,If I Am a Stranger ( Live In Amsterdam ),Ryan Adams,Live After Deaf ( Live ) [ Complete Collection ],"Singer/Songwriter , Music , Rock",$ 0.99,2012 PAXAM RECORDS,5:10,2-Jul-12,Dear Chicago ( Live in Stockholm ),Ryan Adams,Live After Deaf ( Collection ),"Folk , Rock",$ 0.99,2012 PAXAM RECORDS,2:29,"July 2 , 2012"
155,0.067019,1285,34041,0,I 'm a Machine ( feat . Crystal Nicole and Tyr...,David Guetta,Nothing But the Beat,"Dance , Music , Rock , House , Electronic",$ 1.29,"2011 What A Music Ltd , Licence exclusive Parl...",3:34,26-Aug-11,Little Bad Girl ( feat . Taio Cruz & Ludacris ...,David Guetta,Nothing But the Beat 2.0 [ Explicit ],Dance & Electronic,$ 0.99,2015 Real Sounds,3:12,"August 26 , 2011"
